In [ ]:
#BASE IDEA

N_BLOCKS = 100

w3 = Web3(Web3.HTTPProvider("https://still-powerful-cherry.discover.quiknode.pro/d19f575699ccc200b61ee6d889113e34a1797e62/"))
latest_block_number = w3.eth.block_number
N_BLOCKS = latest_block_number - N_BLOCKS

def get_block(block_number):
    
    block = w3.eth.get_block(block_number, full_transactions=True)
    block = dict(block)
    return(block)
blocks = []
for n in range(N_BLOCKS, latest_block_number):
    
    blocks.append(get_block(n))
print(blocks[0])


In [ ]:
import json
import sqlite3
from web3 import Web3
from web3.datastructures import AttributeDict
from hexbytes import HexBytes

N_BLOCKS = 20

w3 = Web3(Web3.HTTPProvider("https://still-powerful-cherry.discover.quiknode.pro/d19f575699ccc200b61ee6d889113e34a1797e62/"))
latest_block_number = w3.eth.block_number
N_BLOCKS = latest_block_number - N_BLOCKS

# Set up the SQLite database
conn = sqlite3.connect('./db/block1.db')
c = conn.cursor()

# Create tables
c.execute('''
CREATE TABLE IF NOT EXISTS blocks
(id TEXT PRIMARY KEY, baseFeePerGas TEXT, difficulty TEXT, extraData TEXT, gasLimit TEXT, 
gasUsed TEXT, hash TEXT, logsBloom TEXT, miner TEXT, mixHash TEXT, nonce TEXT, 
number TEXT, parentHash TEXT, receiptsRoot TEXT, sha3Uncles TEXT, size TEXT, 
stateRoot TEXT, timestamp TEXT, totalDifficulty TEXT, transactionsRoot TEXT, 
uncles TEXT, withdrawals TEXT, withdrawalsRoot TEXT)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS transactions
(id INTEGER PRIMARY KEY, block_id TEXT, tx_index INTEGER, transaction_data TEXT)
''')

def get_block(block_number):
    
    block = w3.eth.get_block(block_number, full_transactions=True)
    block = dict(block)
    
    # Insert block data into the blocks table
    c.execute('''
    INSERT OR IGNORE INTO blocks VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (str(block_number), str(block['baseFeePerGas']), str(block['difficulty']), str(block['extraData']), str(block['gasLimit']), str(block['gasUsed']), str(block['hash'].hex()), 
          str(block['logsBloom'].hex()), str(block['miner']), str(block['mixHash'].hex()), str(block['nonce'].hex()), str(block['number']), str(block['parentHash'].hex()), str(block['receiptsRoot'].hex()), 
          str(block['sha3Uncles'].hex()), str(block['size']), str(block['stateRoot'].hex()), str(block['timestamp']), str(block['totalDifficulty']), str(block['transactionsRoot'].hex), 
          str(block['uncles']), str(block['withdrawals']), str(block['withdrawalsRoot'])))
    
    # Insert each transaction into the transactions table
    for tx_index, tx in enumerate(block['transactions']):
        tx = {k: v.hex() if isinstance(v, HexBytes) else str(v) for k, v in tx.items()}
        c.execute('''
        INSERT OR IGNORE INTO transactions VALUES (?, ?, ?, ?)
        ''', (None, str(block_number), tx_index, str(tx)))
    return block
blocks = []
for n in range(N_BLOCKS, latest_block_number):
    blocks.append(get_block(n))

# Commit the changes and close the connection
conn.commit()
conn.close()

print(blocks)


In [ ]:

def get_block_by_hash(block_hash):
    
    conn = sqlite3.connect('./db/block1.db')
    c = conn.cursor()
    c.execute('''
    SELECT * FROM blocks WHERE hash = ?
    ''', (block_hash,))
    result = c.fetchone()
    conn.close()

    return result

def get_transaction(transaction_hash):
    conn = sqlite3.connect('./db/block1.db')
    c = conn.cursor()
    c.execute('''
    SELECT * FROM transactions WHERE transaction_data LIKE ?
    ''', ('%{}%'.format(transaction_hash),))
    result = c.fetchone()
    conn.close()

    return result

block_hash = "0x3be81d14b10e19508e4dba2df7ee4a194ba9d9b85559e81600bd9a5ac53a94e2"
transaction_hash = "0xfbcb6c9694ffd2bb7d79b15a711802df96022e1b98ba9a31f53e6791daf73a50"

print(get_block(block_hash))
print(get_transaction(transaction_hash))


In [ ]:
#api proto
from flask import Flask, jsonify
import sqlite3

app = Flask(__name__)

def get_block(block_hash):
    conn = sqlite3.connect('./db/block1.db')
    c = conn.cursor()

    c.execute('''
    SELECT * FROM blocks WHERE hash = ?
    ''', (block_hash,))

    result = c.fetchone()

    conn.close()

    return result

def get_transaction(transaction_hash):
    conn = sqlite3.connect('block1.db')
    c = conn.cursor()

    c.execute('''
    SELECT * FROM transactions WHERE transaction_data LIKE ?
    ''', ('%{}%'.format(transaction_hash),))

    result = c.fetchone()

    conn.close()

    return result

@app.route('/block/<block_hash>', methods=['GET'])
def block_route(block_hash):
    block = get_block(block_hash)
    if block is not None:
        return jsonify(block)
    else:
        return jsonify({"error": "Block not found"}), 404

@app.route('/transaction/<transaction_hash>', methods=['GET'])
def transaction_route(transaction_hash):
    transaction = get_transaction(transaction_hash)
    if transaction is not None:
        return jsonify(transaction)
    else:
        return jsonify({"error": "Transaction not found"}), 404

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
#Debug

# Re-open the SQLite database
conn = sqlite3.connect('./db/block1.db')
c = conn.cursor()

# Fetch the first 5 rows from the blocks table
c.execute('''
SELECT * FROM blocks LIMIT 5
''')
print("First 5 rows from the blocks table:")
for row in c.fetchall():
    print(row)

# Fetch the first 5 rows from the transactions table
c.execute('''
SELECT * FROM transactions LIMIT 5
''')
print("\nFirst 5 rows from the transactions table:")
for row in c.fetchall():
    print(row)

# Close the connection
conn.close()
#10 12 19